# Part 2
In the first part, we have seen how to compute joint probability of a sentence using the independence assumption.
However, this assumption is too weak as words do depend on each other. In the second part, we would like to introduce a better approach using Markov chain.

#### Markov chain

Markov chain is based on the following assumption: *given the present, the future does not depend on the past*.

##### Zeroth order Markov chain
The simplest form of Markov chain is so called zeroth order Markov chain where words are completely independent from each other:

$p(w_1\ w_2\ \dots\ w_{|T|}) \approx p(w_1)\ p(w_2)\ p(w_3)\ \dots \ p(w_{|T|}) = \prod^{|T|}_{i=1}p(w_i)$

Recall that this is similar to the independence assumption covered in the first part.
In literature, this probabilistic model is also known as unigram language model.

##### First order Markov chain:
In the first order Markov chain, a probability of a word is conditioned only on the preceeding word:

$p(w_1\ w_2\ \dots\ w_{|T|}) \approx p(w_1)\ p(w_2|w_1)\ p(w_2|w_3)\ \dots \ p(w_{|T|}|w_{|T|-1}) = p(w_1)\prod^{|T|}_{i=2}p(w_i|w_{i-1})$

where conditional probabilities are estimated as:

$p(w_i|w_{i-1})=\frac{count(w_{i-1}\ w_{i})}{count(w_{i-1})}$

In literature, this probabilistic model is called bigram (or 2-gram) language model.

Similarly, in the N-th order Markov chain, a probability of a word is conditioned only on the preceeding N words.


For each of the following tasks we will continue working with PTB dataset.

### Task 1: Computing probability of a sentence (first order Markov chain)
Write a function called **compute_prob1** which takes a sentence as input (and maybe some additional arguments) and returns its probability estimated using the first order Markov chain. 

Your function should first append **\<bos>** and **\<eos>** symbols to the input sentence  and replace all words that are not present in the dictionary with **\<unk>** symbol. 

If the original sentence was "my name is madina", then "madina" is obviously not in the dictionary (see the description of the PTB). The probability of such sentence can be computed as:
  p("my name is madina") = p("my" | "\<bos>") * p("name" | "my") * p("is" | "name") * p("\<unk>" | "is") * p("\<eos>" | "\<unk>")
  

If you completed the task correctly then your function should compute:

p("my name is madina") = 8.512130344833542e-10

p("hello how are you") = 6.89010961921847e-13

p("this is an estimate") = 4.418442586905126e-10

In [1]:
import numpy as np
import random
text = open('ptb.train.txt', encoding='utf8').read()
data = text.split('\n')
data = [sent for sent in data if sent != '']
for i in range(len(data)):
    data[i] = ''.join(('<bos>', data[i],'<eos>'))
    
d = []
for string in data:
    s = string.split()
    for e in s:
        d.append(e)

dictionary = dict()
for w in d:
    if w in dictionary:
        dictionary[w] += 1
    else:
        dictionary[w] = 1

s = sum(dictionary.values())
dictionary_prob = dict()
for i in dictionary:
    dictionary_prob[i] = dictionary[i]/s

In [145]:
def compute_prob1(s, d, dct):
    s = ''.join(('<bos>', ' ', s, ' ', '<eos>'))
    s = s.split(' ')
    for i in range(len(s)):
        if s[i] not in d:
            s[i] = "<unk>"
            
    p = 1 
    for i in range(len(s)-1):
        count = 0
        for j in range(len(d)-1):
            if (s[i] == d[j]) & (s[i+1] == d[j+1]):
                count += 1
        if count > 0:
            p *= count/dct[s[i]]                
    return p

for i in range(3):
    sent = input()
    print(compute_prob1(sent, d, dictionary), '\n')

my name is madina
8.512130344833542e-10 

hello how are you
6.89010961921847e-13 

this is an estimate
4.418442586905126e-10 



### Task 2: Predicting the most likely next words (Autocomplete)
In this task, you will implement autocomeplete function which is similar to the message completion service in mobile phones.

Write a function **autofill** which takes a word and integer k as input (and maybe some additional arguments) and returns k most probable words to follow it according to the estimates computed by first order Markov chain.

If you completed the task correctly, your function should return that the 5 most probable words to follow "san" are \['francisco', '\<unk>', 'jose', 'diego', 'antonio'\]

In [4]:
def autofill(n, w, dt):
    nex = dict()
    for i in range(len(dt)-1):
        if dt[i] == w:
            if dt[i+1] in nex:
                nex[dt[i+1]] += 1
            else:
                nex[dt[i+1]] = 1
    
    sortedAns = sorted(nex, key=nex.get, reverse = True)
    
    if len(sortedAns) >= int(n):
        return sortedAns[0:int(n)]
    else:
        return "Try another number"

num = input()
word = input()
autofill(num, word, d)

5
san


['francisco', '<unk>', 'jose', 'diego', 'antonio']

### Task 3: Generate a text (zeroth order Markov chain)
Write a function **generate_text0** which takes as an input integer k (and maybe some additional arguments) and generates k sentences sampled from the probability distribution estimated by the zeroth order Markov chain. 

The length of each sentence should be at least 3 words, including \<bos> and \<eos>. For example "\<bos> is \<eos>".

\<bos> and \<eos> must not appear in the middle of a sentence.

Hint: you can use random.choices() for sampling.

If you completed the task correctly, then for k = 2 your output would look like this (but with different sentences):

['\<bos> fees property the year 's drop world died or \<unk> j. trust the which for meanwhile action economic criminal germany \<unk> by with white \<eos>',

 '\<bos> koch to a N laws \<eos>']

In [8]:
def generate_text0(n, dat): 
   
    for i in range(int(n)):
        choose = np.random.choice(dat)
        if choose != '<bos>' and choose != '<eos>':
            first_word = choose
        sentence = [first_word]
        n_words = random.randint(1, 7)
    
        for i in range(n_words):
            w = np.random.choice(dat)
            if w != '<bos>' and w != '<eos>':
                sentence.append(w)
        sentence = ' '.join(sentence)
        print(' '.join(('<bos>', sentence,'<eos>')))
    
print("Enter number of sentences: ")
k = input()
generate_text0(k, d)

Enter number of sentences: 
10
<bos> or he N <unk> <eos>
<bos> consistently improving case <eos>
<bos> fairly remarks tv <eos>
<bos> dec. reflecting bond this the <unk> ' <eos>
<bos> this and some star years leading <eos>
<bos> half inc. partnership they <eos>
<bos> from since jenrette as australia if give <eos>
<bos> the asking 1980s hit for raised but over <eos>
<bos> donald agency anheuser after volume of instruction company <eos>
<bos> program too <unk> $ <eos>


### Task 4: Generate a text (first order Markov chain)
Write a function **generate_text1** which takes as an input integer k (and maybe some additional arguments) and generates k sentences sampled from the probability distribution estimated using the first order Markov chain.

The length of each sentence should be at least 3 words, including \<bos> and \<eos>. For example "\<bos> is \<eos>".

\<bos> and \<eos> must not appear in the middle of a sentence.

Hint: you can use random.choices() for sampling.

If you completed the task correctly, then for k = 2 your output would look like this (but with different sentences):

['\<bos> what happened at st. louis assembly business \<eos>',

 '\<bos> integrated combines some people familiar with forecasts \<eos>',

In [67]:
def generate_text1(n, dat): 
    
    def create_dict(a):
        for i in range(len(a) - 1):
            yield (a[i], a[i + 1])
    chain_dict = create_dict(dat)

    word_dict = {}
    for present, future in chain_dict:
        if present in word_dict.keys():
            word_dict[present].append(future)
        else:
            word_dict[present] = [future]
   
    for i in range(int(n)):
        choose = np.random.choice(dat)
        if choose != '<bos>' and choose != '<eos>':
            first_word = choose
        sentence = [first_word]
        n_words = random.randint(1, 7)
    
        for i in range(n_words):
            w = np.random.choice(word_dict[sentence[-1]])
            if w != '<bos>' and w != '<eos>':
                sentence.append(w)
        sentence = ' '.join(sentence)
        print(' '.join(('<bos>', sentence,'<eos>')))
    
print("Enter number of sentences: ")
k = input()
generate_text1(k, d)

Enter number of sentences: 
2
legal
<unk>
<bos> conservative legal <unk> <eos>
until
he
<unk>
<unk>
messrs.
watson
<bos> <unk> until he <unk> <unk> messrs. watson <eos>


In [68]:
def create_dict(d):
    for i in range(len(d) - 2):
        yield (d[i], d[i + 1], d[i + 2])
chain_dict = create_dict(d)

word_dict = {}

for present, present2, future in chain_dict:
    dkey = [present, present2]
    if tuple(dkey) in word_dict.keys():
        word_dict[tuple(dkey)].append(future)
    else:
        word_dict[tuple(dkey)] = [future]

In [165]:
def generate_text2(n, dat): 
    
    def create_dict(d):
        for i in range(len(d) - 2):
            yield (d[i], d[i + 1], d[i + 2])
    chain_dict = create_dict(d)

    word_dict = {}

    for present, present2, future in chain_dict:
        dkey = [present, present2]
        if tuple(dkey) in word_dict.keys():
            word_dict[tuple(dkey)].append(future)
        else:
            word_dict[tuple(dkey)] = [future]
   
    for i in range(int(n)):
        first_word = []
        index = random.randint(0, len(d) - 1)
        key = (d[index], d[index + 1])
        if '<bos>' not in key and '<eos>' not in key:
            first_word = key        
        sentence = list(first_word)
        n_words = random.randint(1, 7)
    
        for i in range(n_words):             
            w = np.random.choice(word_dict[key])
            if w != '<bos>' and w != '<eos>':
                sentence.append(w)
            key = (key[1], w)
        output = ' '.join(sentence)
        print(' '.join(('<bos>', output,'<eos>')))

In [168]:
print("Enter number of sentences: ")
k = input()
generate_text2(k, d)

Enter number of sentences: 
1
('article', 'ii')
('ii', 'of')
('of', 'the')
('the', '<unk>')
('<unk>', 'countries')
<bos> of article ii of the <unk> countries <eos>


In [167]:
def generate_text2(n, dat): 
    
    def create_dict(d):
        for i in range(len(d) - 2):
            yield (d[i], d[i + 1], d[i + 2])
    chain_dict = create_dict(d)

    word_dict = {}

    for present, present2, future in chain_dict:
        dkey = [present, present2]
        if tuple(dkey) in word_dict.keys():
            word_dict[tuple(dkey)].append(future)
        else:
            word_dict[tuple(dkey)] = [future]
   
    for i in range(int(n)):
        first_word = []
        index = random.randint(0, len(d) - 1)
        key = (d[index], d[index + 1])
        if '<bos>' not in key and '<eos>' not in key:
            first_word = key        
        sentence = list(first_word)
        n_words = random.randint(1, 7)
    
        for i in range(n_words):             
            w = np.random.choice(word_dict[key])
            if w != '<bos>' and w != '<eos>':
                sentence.append(w)
            key = (key[1], w)
            print(key)
        output = ' '.join(sentence)
        print(' '.join(('<bos>', output,'<eos>')))